In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from keras import applications
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img,array_to_img
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras import backend as K
K.set_image_dim_ordering('th')

import progressbar

import os

from IPython.display import Image, display, clear_output

In [ ]:
os.chdir("/home/dwx/Documents/Studium/DTU/11/DL Deep Learning/project/ISIC_MSK-2_1_sorted")

In [ ]:
def get_model(top_model_weights_path, img_width, img_height):
    input_tensor = Input(shape=(3,img_width,img_height))
    base_model = applications.VGG16(weights='imagenet',include_top= False,input_tensor=input_tensor)
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(0.5))
    top_model.add(Dense(1, activation='sigmoid'))
    #top_model.load_weights(top_model_weights_path)
    model = Model(inputs= base_model.input, outputs= top_model(base_model.output))
    model.load_weights(top_model_weights_path)
    return model


In [ ]:
def predict_image_class(model, file, img_width,img_height):
    #model = applications.VGG16(include_top=False, weights='imagenet')
    x = load_img(file, target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    array = model.predict(x)
    class_predicted = model.predict(array)
    if class_predicted==1:
        print("malignant")
    else:
        print("benign")

In [ ]:
train_data_dir = 'train'
validation_data_dir = 'validation'
test_dir = 'test'

mal_dir = "Malignant"
ben_dir = "Benign"

# path to the model weights files.
top_model_weights_path = '../bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

model = get_model(top_model_weights_path, img_width, img_height)

for trainValTest in [test_dir]:
    actual_mal = [0,0] # true, false
    actual_ben = [0,0] # false, true
    actual_mal_pred = []
    actual_ben_pred = []
    
    for malBen in [mal_dir,ben_dir]:
        cur_path = trainValTest + "/" + malBen + "/"
        for file in progressbar.log_progress(os.listdir(cur_path)[:]):
            
            if file.endswith(".jpg"):
                #predict_image_class(model, path_cur_directory+file, img_width,img_height)

                path_file = cur_path + file
                x = load_img(path_file, target_size=(img_width,img_height))
                x = img_to_array(x)
                x = np.expand_dims(x, axis=0)
                value_prediction = model.predict(x)[0][0]
                
                if malBen == mal_dir:
                    actual_mal_pred += value_prediction
                    if value_prediction > 0.5:
                        actual_mal[0] += 1
                    else:
                        actual_mal[1] += 1
                else:
                    actual_ben_pred += value_prediction
                    if value_prediction > 0.5:
                        actual_ben[0] += 1
                    else:
                        actual_ben[1] += 1
    print(actual_mal)
    print(actual_ben)


In [ ]:
plt.scatter(np.arange(0,len(actual_mal_pred)), actual_mal_pred)
plt.show()
plt.scatter(np.arange(0,len(actual_ben_pred)), actual_ben_pred)
plt.show()